In [17]:
## load dataframe from csv 

import pandas as pd 
import numpy as np 
import os
import matplotlib.pyplot as plt
from PIL import Image as Image2
import scipy.stats


A Messy Notebook for Working with Data split Dataframes (various tasks)

In [5]:


fname = '/home/gsantiago/summer_bee_data/open_sets/open_max_ids_batch2/summer_bee_dataset_open_train_bee_balanced_batch2_sample_num_max.csv'
dft = pd.read_csv(fname)


df2 = pd.read_csv('/home/gsantiago/summer_bee_data/open_sets/open_max_ids_batch1/summer_bee_dataset_open_train_bee_balanced_batch1_sample_num_max.csv')
#print(np.unique(dft['label']))


dft = pd.concat((dft,df2))
dft_sorted = dft.sort_values(by='color_id')

In [12]:
b1_to_b2 = {}
b2_to_b1 = {}
for color in np.unique(dft['color_id']):
    cs = list(np.unique(dft[dft['color_id']==color]['ID']))
    if len(cs)>1:
        b1_to_b2[cs[0]] = cs[1]
        b2_to_b1[cs[1]]= cs[0]
    else:
        b1_to_b2[cs[0]] = cs[0]
        b2_to_b1[cs[0]]= cs[0]

print(b1_to_b2)




{10: 74, 11: 75, 15: 79, 14: 78, 12: 76, 16: 80, 13: 77, 9: 73, 18: 82, 19: 83, 23: 87, 22: 86, 20: 84, 24: 88, 21: 85, 17: 81, 50: 114, 51: 115, 55: 119, 54: 118, 52: 116, 56: 120, 53: 117, 49: 113, 42: 106, 43: 107, 47: 111, 46: 110, 44: 108, 48: 112, 45: 109, 41: 105, 26: 90, 27: 91, 31: 95, 30: 94, 28: 92, 32: 96, 29: 93, 25: 89, 58: 122, 59: 123, 63: 127, 62: 126, 60: 124, 64: 128, 61: 125, 57: 121, 34: 98, 35: 99, 39: 103, 38: 102, 36: 100, 40: 104, 37: 101, 33: 97, 2: 66, 3: 67, 7: 71, 6: 70, 68: 68, 8: 72, 5: 69, 1: 65}


In [24]:
filename = '/home/gsantiago/summer_bee_data/open_sets/new_open_max_ids_batch2/summer_bee_dataset_open_train_bee_64_ids_batch2_sample_num_max.csv'

in_df = pd.read_csv(filename)
import pandas as pd

del in_df['Unnamed: 0']
del in_df['Unnamed: 0.1']
del in_df['Unnamed: 0.2']
#del in_df['Unnamed: 0.3']
#del in_df['Unnamed: 0.4']



def extract_and_delete_rows(df, id_column,num_rows):
    unique_ids = df[id_column].unique()
    
    new_df = pd.DataFrame(columns=df.columns)

    for unique_id in unique_ids:
        mask = df[id_column] == unique_id
        first_row = df[mask].iloc[0:num_rows]
        new_df = pd.concat([new_df, first_row], ignore_index=True)
        
        # Delete the rows from the original DataFrame
        first_row_index = df[mask].index[0:num_rows]
        df = df.drop(first_row_index)
    
    return new_df, df

# Example usage:
# Assuming 'id_column' is the name of the column you want to use
# and 'original_df' is the DataFrame you are working with
result_df, remaining_df = extract_and_delete_rows(in_df, 'ID',1)
print(len(in_df), len(remaining_df))
out_dir = '/home/lmeyers/summer_bee_data_reextract/'

result_df.to_csv(out_dir+os.path.basename(filename)[:-4]+'.classifer_train_1_img.csv',index=False)
remaining_df.to_csv(out_dir+os.path.basename(filename)[:-4]+'.classifer_test_img.csv',index=False)
remaining_df.head()


/tmp/ipykernel_12451/533246032.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, first_row], ignore_index=True)


10866 10802


,img_number,ID,color_id,frame,track,batch,pass,environment,bee_range,filepath,background,reID,color_num
1,1,74,blue-blue,14363,83,2,NaN,in-lab,65-96,/home/gsantiago/summer_bee_data/summer_bee_dat...,blue,74,0
2,2,74,blue-blue,14067,83,2,NaN,in-lab,65-96,/home/gsantiago/summer_bee_data/summer_bee_dat...,blue,74,0
3,3,74,blue-blue,19930,171,2,2.0,outside,65-96,/home/gsantiago/summer_bee_data/summer_bee_dat...,blue,74,0
4,4,74,blue-blue,19756,166,2,2.0,outside,65-96,/home/gsantiago/summer_bee_data/summer_bee_dat...,blue,74,0
5,5,74,blue-blue,14066,83,2,NaN,in-lab,65-96,/home/gsantiago/summer_bee_data/summer_bee_dat...,blue,74,0


In [29]:
import glob
for f in glob.glob('/home/lmeyers/summer_bee_data_reextract/summer*test*'):
    print(f)
    df = pd.read_csv(f)
    unique_values = np.unique(df['ID'])
    num_rows = np.round(len(df)/len(unique_values)*.15)
    print(num_rows)
    #index_mapping = {value: index for index, value in enumerate(unique_values)}
    # Add a new column 'col2' with the index for each value in 'col1'
    # del df['Class_num']
    # df['class_num'] = df['ID'].map(index_mapping)
    result_df, remaining_df = extract_and_delete_rows(df, 'ID',int(num_rows))
    print(len(df), len(remaining_df))
    out_dir = '/home/lmeyers/summer_bee_data_reextract/'
    result_df.to_csv(out_dir+os.path.basename(f)[:-23]+'.classifer_valid_20_percent.csv',index=False)
    remaining_df.to_csv(out_dir+os.path.basename(f)[:-23]+'.classifer_test_img.csv',index=False)
    # remaining_df.head()

    

/home/lmeyers/summer_bee_data_reextract/summer_bee_dataset_open_train_bee_24_ids_monocolor_and_permutated_ids_batch2_sample_num_max.classifer_test_img.csv
26.0
4226 3602
/home/lmeyers/summer_bee_data_reextract/summer_bee_dataset_open_train_bee_16_ids_four_times_permutated_colors_batch2_sample_num_max.classifer_test_img.csv
28.0


/tmp/ipykernel_12451/533246032.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, first_row], ignore_index=True)
/tmp/ipykernel_12451/533246032.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, first_row], ignore_index=True)


3040 2592
/home/lmeyers/summer_bee_data_reextract/summer_bee_dataset_open_train_bee_32_ids_permutated_ids_batch2_sample_num_max.classifer_test_img.csv
26.0
5631 4803


/tmp/ipykernel_12451/533246032.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, first_row], ignore_index=True)
/tmp/ipykernel_12451/533246032.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, first_row], ignore_index=True)


/home/lmeyers/summer_bee_data_reextract/summer_bee_dataset_open_train_bee_4_ids_batch2_sample_num_max.classifer_test_img.csv
27.0
725 617
/home/lmeyers/summer_bee_data_reextract/summer_bee_dataset_open_train_bee_8_ids_double_colors_batch2_sample_num_max.classifer_test_img.csv
22.0
1191 1015
/home/lmeyers/summer_bee_data_reextract/summer_bee_dataset_open_train_bee_64_ids_batch2_sample_num_max.classifer_test_img.csv
25.0


/tmp/ipykernel_12451/533246032.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, first_row], ignore_index=True)
/tmp/ipykernel_12451/533246032.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, first_row], ignore_index=True)


10802 9205


In [13]:
import pandas as pd 
fn = '/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch2/summer_bee_dataset_open_train_bee_4_ids_batch2_sample_num_64.csv'
df = pd.read_csv(fn)

fn2 = '/home/gsantiago/summer_bee_data/open_sets/new_open_04_ids_all_colors_batch2/summer_bee_dataset_open_train_bee_4_ids_batch2_sample_num_max.csv'
df2 = pd.read_csv(fn2)


print(len(df2))
# # Check if lines of df1 are present in df2
# merged_df = pd.merge(df, df2, how='right', indicator=True)

# # Filter the lines present in both DataFrames
# common_lines = merged_df[merged_df['_merge'] == 'both'].drop('_merge', axis=1)

# # Display the result
# print(common_lines)


729
